<a href="https://colab.research.google.com/github/saurabhIU/Deep-Learning/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Dependencies
import librosa
import librosa.display as disp
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time;

In [12]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test_x_01.wav to test_x_01.wav
Saving test_x_02.wav to test_x_02.wav
Saving train_clean_male.wav to train_clean_male.wav
Saving train_dirty_male.wav to train_dirty_male (1).wav
User uploaded file "test_x_01.wav" with length 145284 bytes
User uploaded file "test_x_02.wav" with length 388752 bytes
User uploaded file "train_clean_male.wav" with length 2522886 bytes
User uploaded file "train_dirty_male.wav" with length 2522898 bytes


In [0]:
s_clean, sr_clean=librosa.load('train_clean_male.wav',sr=None)
S=librosa.stft(s_clean, n_fft=1024, hop_length=512)
s_noisy, sr_noisy=librosa.load('train_dirty_male.wav',sr=None)
X=librosa.stft(s_noisy, n_fft=1024, hop_length=512)

In [0]:
S_absolute = np.abs(S)
X_absolute = np.abs(X)

In [0]:
EPOCHS = 400
X = tf.placeholder("float", [None, 513])
Y = tf.placeholder("float", [None, 513])
def nn_model(x):
  
    layer1 = tf.layers.dense(x, 1024,activation=tf.nn.relu)
    layer2 = tf.layers.dense(layer1, 1024,activation=tf.nn.relu)
    logits = tf.layers.dense(layer2, 513,activation=tf.nn.relu)
    return logits

logits = nn_model(X)
cost = tf.losses.mean_squared_error(Y,logits)
optimizer = tf.train.AdamOptimizer().minimize(cost)
prediction = tf.nn.softmax(logits)
correct_prediction = tf.equal(tf.argmax(prediction, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [50]:
X_absolute.shape

(513, 2459)

In [51]:
sess =  tf.Session() 
    
sess.run(tf.global_variables_initializer())
  
tic = time.time()
for i in range(EPOCHS):
    c,_,acc = sess.run([cost, optimizer, accuracy],feed_dict={X: X_absolute.T, Y: S_absolute.T})
    if i % 100 == 0:
      print(f'Epoch: {i},training accuracy:{acc * 100}')
toc = time.time()
print(f'Time taken for training is {toc-tic}')

Epoch: 0,training accuracy:0.0
Epoch: 100,training accuracy:50.30500292778015
Epoch: 200,training accuracy:53.84302735328674
Epoch: 300,training accuracy:55.754369497299194
Time taken for training is 8.373116970062256


In [0]:
s_clean, sr_clean=librosa.load('train_clean_male.wav',sr=None)
S=librosa.stft(s_clean, n_fft=1024, hop_length=512)